In [2]:
import math
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [7]:
class Simulator:
    def __init__(self):
        pass

    def has_empty_cells(self, board):
        result = True

        if np.any(board == 0) == False:
            result = False

        return result
    
    def find_empty_cells(self, board):
        empty_cells = []

        if self.has_empty_cells(board) == True:
            for i in range(3):
                for j in range(3):
                    if board[i,j] == 0:
                        empty_cells.append((i,j))

        return empty_cells
    
    def get_reward(self, board):
        reward = 0
        if self.has_agent_won(board) == True:
            reward = 10
        elif self.has_enemy_won(board) == True:
            reward = -10
        elif self.has_game_drawn(board) == True:
            reward = 0
        else:
            reward = 1

        return reward

    def has_agent_won(self, board):
        result = False

        row_wise_sum = np.sum(board, axis=0)
        col_wise_sum = np.sum(board, axis=1)
        main_diag_sum = np.sum([board[i,i] for i in range(3)])
        off_diag_sum = np.sum([board[i,2-i] for i in range(3)])

        if np.any(row_wise_sum == 3) or np.any(col_wise_sum == 3) or main_diag_sum == 3 or off_diag_sum == 3:
            result = True

        return result

    def has_enemy_won(self, board):
        result = False

        row_wise_sum = np.sum(board, axis=0)
        col_wise_sum = np.sum(board, axis=1)
        main_diag_sum = np.sum([board[i,i] for i in range(3)])
        off_diag_sum = np.sum([board[i,2-i] for i in range(3)])

        if np.any(row_wise_sum == -3) or np.any(col_wise_sum == -3) or main_diag_sum == -3 or off_diag_sum == -3:
            result = True

        return result

    def has_game_drawn(self, board):
        result = False

        if self.has_empty_cells(board) == False and self.has_agent_won(board) == False and self.has_enemy_won(board) == False:
            result = True

        return result
    
    def has_game_end(self, board):
        result = False

        if self.has_agent_won(board) == True or self.has_enemy_won(board) == True or self.has_game_drawn(board) == True:
            result = True

        return result
    
    # returns the updated board state and a bool specifying whether enemy moved or not
    def enemy_move(self, board, empty_cell):
        has_enemy_moved = False

#         if self.has_empty_cells(board) == False:
#             return board, has_enemy_moved

        # find empty cells
#         empty_cells = self.find_empty_cells(board)
#         num_empty_cells = len(empty_cells)

        # generate a random number between [0, num_empty_cells-1] (both inclusive)
#         rand_num = np.random.randint(0, num_empty_cells)

#         enemy_move_pos = empty_cells[rand_num]

        board[empty_cell[0], empty_cell[1]] = -1
        has_enemy_moved = True

        return has_enemy_moved, board
    
    # returns the updated board state and a bool specifying whether enemy moved or not
    def agent_move(self, board, empty_cell):
        has_agent_moved = False

        board[empty_cell[0], empty_cell[1]] = 1
        has_agent_moved = True

        return has_agent_moved, board

    def compute_reward_to_go(self, traj, gamma=0.9):
        traj_len = len(traj)
        reward_to_go = []

        for i in range(traj_len):
            temp_sum = 0
            for j in range(i, traj_len):
                temp_sum = temp_sum + pow(gamma, j-i) * traj[j].reward

            reward_to_go.append(temp_sum)

        return reward_to_go

    def convert_mat_to_string(self, board):
        result = ''
        for i in range(3):
            for j in range(3):
                if board[i,j] == 0:
                    result = result + '-'
                elif board[i,j] == 1:
                    result = result + 'o'
                elif board[i,j] == -1:
                    result = result + 'x'
        return result

    def convert_string_to_mat(self, mystring):
        board = np.zeros((3,3))
        itr = 0
        for i in range(3):
            for j in range(3):
                if mystring[itr] == '-':    
                    board[i,j] = 0
                elif mystring[itr] == 'o':
                    board[i,j] = 1
                elif mystring[itr] == 'x':    
                    board[i,j] = -1
                itr += 1
        return board

In [8]:
global_sim_obj = Simulator()

In [9]:
state = np.array([[1,-1,1],[-1,0,0],[0,0,0]])
a = global_sim_obj.convert_mat_to_string(state)
print(a)
print(global_sim_obj.convert_string_to_mat(a))

oxox-----
[[ 1. -1.  1.]
 [-1.  0.  0.]
 [ 0.  0.  0.]]


In [10]:
class Node:
    def __init__(self, state, player_type=0):
        self.state = np.copy(state)      # 3x3 matrix representing the board state

        self.children = []              # to store a list of (action, child_node) tuples

        self.player_type = player_type
        self.reward = global_sim_obj.get_reward(self.state)
        self.values = 0
        self.nodeID = None

    def is_terminal(self):
        if not self.children:
            return True
        else:
            return False

In [15]:
class TicTacToe:
    def __init__(self, root_state):
        self.root = Node(root_state, ENEMY)
#         self.simulator = Simulator()
        self.state_list = []
        self.node_list = []

    def is_node_present(self, root_node, node_state):
        if np.any(node_state - self.root.state) == False:
            print("This should be printed once")
            return False, None
        
        child_list = root_node.children

        if len(child_list) != 0:
            for child in child_list:
                child_node = child[1]
                if np.any(node_state - child_node.state) == False:
                    return True, child_node
                else:
                    if_present, return_node = self.is_node_present(child_node, node_state)
                    if if_present == True:
                        return True, return_node
        
        return False, None          
        
    def build_tree(self, node, player_type, agent_action=None):
        if node is None:
            node = self.root
        
        if global_sim_obj.has_game_end(np.copy(node.state)) == True:
            return
            
        if player_type == ENEMY:
            init_Q_state = np.copy(node.state)
            
            if global_sim_obj.has_empty_cells(init_Q_state) == True:
                empty_cells = global_sim_obj.find_empty_cells(init_Q_state)
                num_empty_cells = len(empty_cells)
                
                for i in range(num_empty_cells):
                    has_enemy_moved, modified_board_state = global_sim_obj.enemy_move(np.copy(init_Q_state), empty_cells[i])
                    assert(np.sum(modified_board_state) == -1)
                    
                    if has_enemy_moved == True:
#                         present, child_node = self.is_node_present(self.root, np.copy(modified_board_state))
                        present = False
                        
                        if present:
                            node.children.append((agent_action, child_node))
                        else:
                            global num_tree_nodes
                            num_tree_nodes += 1
                            if (num_tree_nodes % 10000 == 0):
                                print("Curr Nodes: ", num_tree_nodes)

                            child_node = Node(modified_board_state)
                            node.children.append((agent_action, child_node))

                            self.state_list.append(np.copy(modified_board_state))
                            self.node_list.append(child_node)

                            if global_sim_obj.has_game_end(modified_board_state) == True:
                                pass
                            else:
                                self.build_tree(child_node, player_type=AGENT)

        elif player_type == AGENT:
            init_board_state = np.copy(node.state)
            
            if global_sim_obj.has_empty_cells(init_board_state) == True:
                empty_cells = global_sim_obj.find_empty_cells(init_board_state)
                num_empty_cells = len(empty_cells)

                for i in range(num_empty_cells):
                    # agent will take an action to get to Q-state
                    agent_action = empty_cells[i]
                    has_agent_moved, modified_Q_state = global_sim_obj.agent_move(np.copy(init_board_state), agent_action)
                    assert(np.sum(modified_Q_state) == 0)
                    
                    if has_agent_moved == True:

                        if global_sim_obj.has_game_end(modified_Q_state) == True:
                            if global_sim_obj.has_agent_won(modified_Q_state) == True:
                                child_node = Node(modified_Q_state)
                                node.children.append((agent_action, child_node))
                        else:
                            node.state = np.copy(modified_Q_state)
                            self.build_tree(node, player_type=ENEMY, agent_action=agent_action)
                            node.state = np.copy(init_board_state)
    
    def print_trajectory(self):
        traj = []
        continue_loop = True
        tree_node = self.root

        while continue_loop:
            traj.append(tree_node)
            node_state = tree_node.state

            if tree_node.is_terminal() == False:
                child_list = tree_node.children
                num_child = len(child_list)

                child_element = child_list[np.random.randint(0, num_child)]

                tree_node = child_element[1]
            else:
                if (global_sim_obj.has_agent_won(tree_node.state) == True):
                    print("Agent Won !!!")
                elif (global_sim_obj.has_enemy_won(tree_node.state) == True):
                    print("Enemy Won !!!")
                elif (global_sim_obj.has_game_drawn(tree_node.state) == True):
                    print("Game Drawn !!!")
                else:
                    print("Something is wrong")

                continue_loop = False
        
        reward_to_go = global_sim_obj.compute_reward_to_go(traj)
        traj_len = len(traj)
        for j in range(traj_len):
            print(traj[j].state, ",  R(s_i) = ", global_sim_obj.get_reward(traj[j].state), " G(s_i) = ", round(reward_to_go[j], 2), "\n")

In [16]:
AGENT, ENEMY = 0, 1
game_obj = TicTacToe(np.zeros((3,3)))

In [17]:
num_tree_nodes = 0
game_obj.build_tree(None, player_type=ENEMY)

Curr Nodes:  10000
Curr Nodes:  20000
Curr Nodes:  30000
Curr Nodes:  40000
Curr Nodes:  50000
Curr Nodes:  60000
Curr Nodes:  70000
Curr Nodes:  80000
Curr Nodes:  90000
Curr Nodes:  100000
Curr Nodes:  110000
Curr Nodes:  120000
Curr Nodes:  130000
Curr Nodes:  140000
Curr Nodes:  150000
Curr Nodes:  160000
Curr Nodes:  170000
Curr Nodes:  180000
Curr Nodes:  190000
Curr Nodes:  200000
Curr Nodes:  210000
Curr Nodes:  220000
Curr Nodes:  230000
Curr Nodes:  240000
Curr Nodes:  250000
Curr Nodes:  260000
Curr Nodes:  270000
Curr Nodes:  280000
Curr Nodes:  290000


In [18]:
# show 5 random trajectory
for i in range(5):
    print("Trajectory -", (i+1))
    game_obj.print_trajectory()
    print("\n")

Trajectory - 1
Agent Won !!!
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[ 0. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0. -1.  0.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[ 1. -1.  0.]
 [ 1.  0.  0.]
 [ 0. -1. -1.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[ 1. -1. -1.]
 [ 1.  0.  1.]
 [ 0. -1. -1.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[ 1. -1. -1.]
 [ 1.  1.  1.]
 [ 0. -1. -1.]] ,  R(s_i) =  10  G(s_i) =  10.0 



Trajectory - 2
Agent Won !!!
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[-1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[-1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[-1. -1.  0.]
 [ 0.  0. -1.]
 [ 1.  1.  0.]] ,  R(s_i) =  1  G(s_i) =  10.0 

[[-1. -1.  0.]
 [ 0.  0. -1.]
 [ 1.  1.  1.]] ,  R(s_i) =  10  G(s_i) =  10.0 



Trajectory - 3
Enemy Won !!!
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] ,  R(s_i) =  1  G(s

In [11]:
print("Total number of states in the tree: ", len(game_obj.state_list))
print("Total number of nodes in the tree: ", len(game_obj.node_list))

num_repeated_states = len(game_obj.state_list)
unique_state_list = []
unique_node_list = []

for i in range(num_repeated_states):
    include = True
    num_unique_states = len(unique_state_list)

    if (i % 10000 == 0):
        print("Iteration: ", i, '/', num_repeated_states, ', # Unique States: ', num_unique_states)
    
    for j in range(num_unique_states):
        if np.any(game_obj.state_list[i] - unique_state_list[j]) == False:
            include = False
            break
    
    if include:
        unique_state_list.append(game_obj.state_list[i])
        unique_node_list.append(game_obj.node_list[i])

Total number of states in the tree:  291681
Total number of nodes in the tree:  291681
Iteration:  0 / 291681 , # Unique States:  0
Iteration:  10000 / 291681 , # Unique States:  716
Iteration:  20000 / 291681 , # Unique States:  963
Iteration:  30000 / 291681 , # Unique States:  1027
Iteration:  40000 / 291681 , # Unique States:  1466
Iteration:  50000 / 291681 , # Unique States:  1652
Iteration:  60000 / 291681 , # Unique States:  1724
Iteration:  70000 / 291681 , # Unique States:  1937
Iteration:  80000 / 291681 , # Unique States:  2134
Iteration:  90000 / 291681 , # Unique States:  2180
Iteration:  100000 / 291681 , # Unique States:  2288
Iteration:  110000 / 291681 , # Unique States:  2419
Iteration:  120000 / 291681 , # Unique States:  2448
Iteration:  130000 / 291681 , # Unique States:  2469
Iteration:  140000 / 291681 , # Unique States:  2572
Iteration:  150000 / 291681 , # Unique States:  2612
Iteration:  160000 / 291681 , # Unique States:  2622
Iteration:  170000 / 291681 , #

In [212]:
num_unique_states = len(unique_state_list)
for i in range(num_unique_states):
    assert(not np.any(unique_state_list[i] - unique_node_list[i].state))

In [284]:
def value_iteration(state_list, node_list, gamma=0.9, epsilon=1e-1):
    delta_thresh = epsilon * (1-gamma) / gamma

    num_states = len(state_list)
    new_values_list = np.zeros((num_states, 1))
    curr_values_list = np.zeros((num_states, 1))

    has_converged = False

    itr_count = 0
    while has_converged == False:
        curr_values_list = np.copy(new_values_list)
        delta = 0
        
        for i in range(num_states):
            board_state = state_list[i]
            state_reward = game_obj.simulator.get_reward(board_state)
            child_list = node_list[i].children
            num_child = len(child_list)
            assert(np.any(board_state - node_list[i].state) == False)
#             print("Assertion successful !!!")
#             print("State: ", board_state)
#             print("Num child: ", num_child)
            
            if num_child > 0:
                child_dict = {}
                
                for j in range(num_child):
                    child_obj = child_list[j]    # (action, child node)
                    child_parent_action = child_obj[0]
                    x,y = child_parent_action

                    if (x,y) not in child_dict:
                        child_dict[(x,y)] = []
#                         print("X & Y: ", x, y)
                    
                    child_node = child_obj[1]
#                     print("Child State:", child_node.state)
                    child_dict[(x,y)].append(child_node)
                    
                actions = list(child_dict.keys())
                num_actions = len(actions)
#                 print("Actions: ", actions, "Num Actions: ", num_actions)
                
                prob = {}
                for j in range(num_actions):
                    action = actions[j]
                    x, y = action
                    num_child_per_action = len(child_dict[(x,y)])
                    prob[(x,y)] = 1 / num_child_per_action
                
#                 print("Probability: ", prob)
                
                max_expected_val = -math.inf
                for j in range(num_actions):
                    action = actions[j]
                    x,y = action
                    pot_states_list = child_dict[(x,y)]
                    num_pot_states = len(pot_states_list)
                    
                    expected_val = 0
                    for k in range(num_pot_states):
                        pot_state = pot_states_list[k].state
                        # search the index of this state in the state list
                        
                        pot_idx = None
                        for itr in range(num_states):
#                             print("This is USA: ", state_list[itr])
#                             print("This is India: ", pot_state)
                            if (np.any(state_list[itr] - pot_state) == False):
                                pot_idx = itr
                                break
                        
                        expected_val = expected_val + prob[(x,y)] * curr_values_list[pot_idx,0]
                    
                    if expected_val > max_expected_val:
                        max_expected_val = expected_val
                
                new_values_list[i,0] = state_reward + gamma * max_expected_val
            else:
                new_values_list[i,0] = state_reward
        
        delta = np.max(np.abs(new_values_list - curr_values_list))
        
        if (delta < delta_thresh):
            print("CONVERGED !!!")
            has_converged = True
        
        itr_count += 1
        print("Iteration -", itr_count, " Delta: ", delta)

    return new_values_list

In [ ]:
value_iteration(game_obj.state_list, game_obj.node_list)
# value_iteration(unique_state_list, unique_node_list)

Iteration - 1  Delta:  10.0
Iteration - 2  Delta:  9.0
